In [72]:
%%time
import numpy as np
import pandas as pd
import string
import matplotlib.pyplot as plt
from pylab import *
from skimage import data, color,io, exposure
from skimage.filters import gaussian,threshold_otsu,threshold_yen,rank
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square,erosion,opening,remove_small_objects,remove_small_holes,watershed,convex_hull_image,disk
from skimage.feature import peak_local_max, canny, blob_dog
from skimage.transform import rotate,hough_circle, hough_circle_peaks
from scipy import ndimage as ndi
import glob, os
import xlsxwriter
from tempfile import TemporaryFile
from ipywidgets import widgets


def colonies(file,cutt=10,Debug=False):
    Image = io.imread(file)
    w,h=Image.shape
    maxi=np.array(np.max(np.max(Image,axis=1),axis=0), dtype=float)
    coin=np.array([np.array(Image[0,0]),np.array(Image[w-1,0]),np.array(Image[0,h-1]),np.array(Image[w-1,h-1])], dtype=float)
    j255=maxi-np.max(coin,axis=0)
    p3=np.percentile(Image+j255,30)
    p8=np.percentile(Image+j255,80)
    IMAGE1=exposure.rescale_intensity(Image+j255, in_range=(p3,p8), out_range=(0,255))
    g1=[0,int(w/2)-int(w/9)]
    g1_bis=[int(w/2)+int(w/9),w]
    g2=[0,int(h/3)-int(h/13),int(h/3)*2-int(h/13)]
    g2_bis=[int(h/3)+int(h/13),int(h/3)*2+int(h/13),h]
    devi=((5,5),(5,0),(5,-5),(-5,5),(-5,0),(-5,-5))*8
    j=0
    w=0
    if Debug==False :
        fig, axes=plt.subplots(nrows=2,ncols=3)
        toto=[]
        book = xlsxwriter.Workbook(file+"_res.xlsx")
        sheet1 = book.add_worksheet('sheet1')
        
    for k in range(2):
        for l in range(3):
            IMAGE=IMAGE1[g1[k]:g1_bis[k],g2[l]:g2_bis[l]]
            if Debug==True :
                plt.close()
                plt.imshow(IMAGE)
                plt.show()
            edges = canny(IMAGE, sigma=2, low_threshold=70, high_threshold=220)  
            # Detect two radii
            hough_radii = np.arange(195, 205, 1)
            hough_res = hough_circle(edges, hough_radii)
            # Select the most prominent 5 circles
            accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii,
                                           total_num_peaks=5)
            f1=int(np.median(cy))
            f2=int(np.median(cx))
            rr=150
            f1,f2=f1+devi[j][0],f2+devi[j][1]
            j+=1
            u1,u2=int(f1-rr),int(f2-rr)
            u3,u4=int(f1+rr),int(f2+rr)
            x, y = np.indices((u3-u1+cutt*2, u4-u2+cutt*2))
            x1, y1=(u3-u1)/2+cutt,(u4-u2)/2+cutt
            mask_circle1 = (x - x1)**2 + (y - y1)**2 < (rr)**2
            mask_circle2 = (x - x1)**2 + (y - y1)**2 < (rr-55)**2
            V02=Image[g1[k]:g1_bis[k],g2[l]:g2_bis[l]][u1-cutt:u3+cutt,u2-cutt:u4+cutt]
            V01=IMAGE[u1-cutt:u3+cutt,u2-cutt:u4+cutt]
            I1=V01>threshold_otsu(V01)
            X2=V01>threshold_yen(V01)
            H2=bitwise_xor(I1,X2)
            interee=V01*mask_circle1
            II1=I1*mask_circle1
            II2=H2*mask_circle1
            
            # denoise image
            markers=II1
            markers=gaussian(markers,2)>0.7
            markers = remove_small_objects(markers,25)
            LLA = erosion (markers,disk(2))
            LLA=markers>0
            if Debug==True :
                plt.imshow(LLA)
                plt.show()
            markers=label(LLA)
            
            if np.max(markers)==0:
                if Debug==True :
                    print('pb')
                if Debug==False :
                    plt.subplot(2,3,j)
                    plt.subplots_adjust(left=None, bottom=None, right=None, top=None,wspace=0, hspace=0.5)
                    img2=plt.imshow(markers,cmap="Paired")
                    img1=plt.imshow(interee,cmap="Greys",alpha=0.6)
                    plt.xticks([])
                    plt.yticks([])
                    sheet1.write(0,w,0)
                    toto=np.append(toto,markers)
                    plt.title(("number of colonies : \n"+str(0)),fontsize=2)
                    w+=1
            if np.max(markers)>0:
                if Debug==True :
                    print('no pb')
                atoi=[]
                for m in range(np.max(markers)):
                        gty=np.count_nonzero(markers==m+1)
                        atoi=np.append(atoi,gty)
                if np.std(atoi)>95000:
                    if Debug==True :
                        print("Low")
                        
                    # denoise image
                    denoised = rank.median(V01, disk(3))
                    if Debug==True :
                        plt.imshow(denoised)
                        plt.show()
                    markers = rank.gradient(denoised, disk(3)) < 15
                    markers = ndi.label(markers)[0]
                    markers = remove_small_objects(markers,300)
                    markers = clear_border(markers,2)
                    markers = remove_small_holes(markers,400)
                    markers = erosion(markers,disk(1))
                    LLA=markers>0
                    lla=label(LLA)
                    erosi=erosion(invert(II2),disk(6))*lla
                    JE=remove_small_objects(erosi,400)*mask_circle2
                    je=np.unique(JE)
                    for XR in je:
                        np.place(lla, lla==XR, 0)
                    lla=lla>0
                    lla=(gaussian(lla,4)>0.08)*mask_circle1
                    if Debug==True :
                        plt.imshow(lla)
                        plt.show()
                    blobs_dog = blob_dog(lla, max_sigma=30, threshold=.1)
                    blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2)
                    Blob=np.where(blobs_dog[:, 2]>=3)[0]
                    distance = ndi.distance_transform_edt(lla)
                    LLAA=lla*0
                    for xi in Blob:
                        LLAA[int(blobs_dog[xi][0]),int(blobs_dog[xi][1])]=1
                    markers = ndi.label(LLAA)[0]
                    labels = watershed(-distance, markers, mask=lla ,compactness=10)
                    LLA=np.max(labels)
                    Lab=labels  
                    if Debug==True :
                        plt.imshow(labels,cmap="Paired")
                        plt.show()
                        print('Blob =',np.size(Blob))
                        print('final =',LLA)
                    
                if np.std(atoi)<=95000:
                    if Debug==True :
                        print("High")
                    distance = ndi.distance_transform_edt(LLA)
                    blobs_dog = blob_dog(LLA, max_sigma=30, threshold=.1)
                    blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2)
                    Blob=np.where(blobs_dog[:, 2]>=0)[0]
                    LLAA=LLA*0
                    for xi in Blob:
                        LLAA[int(blobs_dog[xi][0]),int(blobs_dog[xi][1])]=1
                    markers = ndi.label(LLAA)[0]
                    labels = watershed(-distance, markers, mask=LLA ,compactness=10)
                    LLA=np.max(labels)
                    Lab=labels
                
                    if Debug==True :
                        plt.imshow(labels,cmap="Paired")
                        plt.show()
                        print('Blob =',np.size(Blob))
                        print('final =',LLA)
                if Debug==False :
                    tmp=[]
                    plt.subplot(2,3,j)
                    plt.subplots_adjust(left=None, bottom=None, right=None, top=None,
                        wspace=0, hspace=0.5)
                    img2=plt.imshow(labels,cmap="Paired")
                    img1=plt.imshow(interee,cmap="Greys",alpha=0.6)
                    plt.xticks([])
                    plt.yticks([])
                    sheet1.write(0,w,LLA.astype(np.float64))
                    for m in range(LLA):
                        gty=np.count_nonzero(labels==m+1)*9254.44
                        sheet1.write(m+1,w,gty)
                        tmp=np.append(tmp,gty)
                    toto=np.append(toto,LLA)
                    plt.title(("number of colonies : \n"+str(LLA)+"\nsize of colonies in microm**2 :\n"+str(tmp)),fontsize=2)
                    w+=1    
    if Debug==False :            
        book.close()
        plt.savefig((file+".png"),dpi=500)
    plt.close()
    return(V01,mask_circle1)


CPU times: user 712 µs, sys: 1.34 ms, total: 2.06 ms
Wall time: 2.08 ms


In [75]:
%%time
dire='/Users/Home/folder/' ### Your Folder
files = []
pattern   = "*.tif"

### find files.
for dir,_,_ in os.walk(dire):
    files.extend(glob.glob(os.path.join(dir,pattern))) 
files=sorted(files)

### run script.
for i in files :
    colonies(i)

CPU times: user 1.09 ms, sys: 1.29 ms, total: 2.39 ms
Wall time: 1.45 ms
